# 发布模型

现在模型应该已经完成了在测试集上的评估,准备发布到生产环境了.


## 向客户解释模型,调整客户预期

工作的成功和客户满意是持续满足或超越客户的期望的结果.现在交付的模型只是一半,另一半是让客户建立合理的预期.

非专业人士对人工智能系统的期望往往过高.他们可能期望人工智能'理解'交付给它的任务,执行任务时具有人类的常识.为了解决这个问题,可能需要向客户展示模型失败的案例.(分类问题的话,可以展示错误分类的结果,特别是那些在人类看来匪夷所思的错误)

客户也可能会期望模型能够达到人类的水准.特别是一些之前原本由人工进行的工作,客户往往会期望模型会至少和人类工作一样好.但是大多数的机器学习模型是由人工标注的标签训练的,这些模型可以接近人类的水平但是没法达到这个水平.我们需要向客户清楚的表达模型的预期.

不要给的抽象的数字,结合落地的业务去说明.模型的准确度是 98%,这个数字很精确,大部分人心理上会将其认为 100%,无意间误导客户.结合具体业务说明,例如谈论到信用卡欺诈,要用到模型的假阳性率或假阴性率假如是 5% 和 2.5%.这样说明: 每天有 200 笔有效交易被误报,需要人工审查.平均 14 笔欺诈交易被忽略,而 266 比欺诈交易被正确报告.

你应该确保和深入业务的人员交流决定模型关键的启动参数,例如交易被标记诈骗的阈值等等,这中间的指标的权衡只有深度理解业务的人才能给出答案.


## 部署模型

虽然 python 是现在深度学习主力的开发语言,但我们无法保证模型运行的实际环境,模型也有可能需要导出到不支持 python 的环境.

其次实际生产环境中,模型只用于输出预测,真正部署时还需要进行各种优化,例如:空间占用 预测速度等等.

下面是一些部署模型的选择


### 模型转换成 REST API 部署

将模型转换成 REST API 是常见的部署方式.服务器或者云上安装 tensorflow 建立 REST API.可以使用 flask 或者使用 TensorFlow Serving 实现(tensorflow 自带的库).[TensorFlow Serving](www.tensorflow.org/tfx/guide/serving) 部署 keras 模型仅需要几分钟.

需要将模型部署成 REST API 时一般在这样几种情况:

- 使用模型的应用程序有可靠的互联网连接.
- 应用没有严格的延迟要求,请求/推理/回答的往返实际可能有 500 ms.
- 发送给模型的输入数据不涉及敏感隐私数据,这些数据最终都会在服务器解密,喂给模型.即使这些数据并不敏感,传输过程中也应该使用全程 https.

典型的适合部署成 REST API 的项目: 图像搜索引擎,音乐推荐,信用卡欺诈检查,卫星图像处理等.

部署 REST API 还有一个重要问题是服务器的归属,服务器可以是自己完全掌控.也可以使用第三方的托管服务.例如 google 的 Cloud AI Platform 可以将你上传的模型运行在 google 云服务器上,并给出 REST API.服务器的运维都是由 google 提供,你只需交钱.


### 在设备上部署

有时会有模型必须运行在设备上的需要.这里的设备可能是智能手机 嵌入式ARM 或者最难的只是个单片机.一个运行在设备的机器学习模型例子,现在手机/相机镜头都有的人脸检测.

在下面情况下,我们需要把模型部署到设备上

- 有严格的延迟限制,或者运行在一个非常难以保证联网的环境.又或者是 vr 应用,部署在云端,那个是过山车了.
- 目标的内存和功率受到了严格限制.模型需要足够小,这种情况下 [最佳化機器學習模型](https://www.tensorflow.org/model_optimization) 会有所帮助
- 精度并不是任务的关键,这种情况下可以在运行效率上进行取舍.
- 模型需要的敏感隐私数据,此时数据是绝对不能也不应该离开用户设备.

一些例子

- spam 检测,大部分都是端到端加密了,只能是在设备本身运行.
- 用于工业设备的模型,工业设备要求的实时 保密 对功耗和内存的限制等等.

当然 google 也想到了智能手机和嵌入式设备,我们的首选方案是 [tensorflow lite](www.tensorflow.org/lite),包括 android ios 和一些 arm64 平台支持(怎么少的了树莓派),它包含有一个转换器,可以将 keras 模型直接转换为 tensorflow lite 格式.


### 部署在浏览器

虽然 REST API 部署有它的好处,但是还需要投入服务器等成本.实际上深度学习模型也能跑在浏览器上,直接调用用户的 cpu gpu,省钱.

下面情况下可以考虑浏览器部署

- 把计算过程交给用户,省下服务器成本.
- 最终数据需要留在用户端,假如用户端可以运行模型,何乐而不为呢?
- 有非常严格的延迟要求,省去网络传输时间.
- 要求可以离线工作,又很难安装应用程序,浏览器是方便的替代.

当然运行在浏览器的模型必须要足够小,不能占用太多用户资源.而且此时模型的全部都是暴露给用户的,部署到浏览器之前,任何你需要保密的部分都需要剥离.

部署到浏览器,实际上就是把模型转换到 [tensorflow.js](www.tensorflow.org/js),几乎所有的 keras api 都有实现.


## 模型规模优化

如果面对资源非常受限的平台,缩减模型大小是最直接的,最有效的手段.

- 权重修剪,只保留对结果影响最大的权重,在精度和模型大小直接取舍.
- 权重量化,深度学习模型的权重基本上都是单精度浮点数 float32 类型,然而将权重量化成 8 位数字(int8)是可能的,这样压缩后模型权重的空间直接压缩 4 倍,但是精度可能会和原始模型非常接近.

当然 tensorflow 也包含了权重修剪和权重量化 [最佳化機器學習模型](https://www.tensorflow.org/model_optimization).与 keras 深度集成.


## 持续监控你的模型

现在我们已经完成了模型开发的一切,也部署模型到了生产环境.

不幸的是部署模型并不是终点,还需要对模型的持续监控,模型在新数据上的表现,模型与应用程序的互动,模型对业务的影响等等.

就像观众也是演出的一部分,当模型部署以后,模型会对环境产生影响.一旦有足够多的新数据,此时就需要对模型进行调整.

如果可能,及时对模型在生产环境的预测进行复核,重复使用相同的数据发送给模型,收集预测结果.如果人工复核不可行时,尝试其他评估途径,问卷调查等等.


## 维护模型

大部分模型都是与人本身有关,而随着时间推移,即使相同的信息,在不同时代也有完全不同的含义.前文已经说过概念漂移了,随着时间推移,数据在不断变化,这在不断降低模型的性能.典型的音乐推荐系统寿命是按照周计算,信用卡欺诈检查是几天,图片搜索引起可能是几年.

一旦模型部署,就要开始下一代的准备工作了

- 注意生产环境数据的变化,如果你能接触到的话
- 继续收集和标注数据,特别是现在模型难以分类的数据,随着时间和自动化的进行,累计的数据将支撑下一代模型的开发.

至此机器学习的通用流程就结束了,成为机器学习专家需要时间,但这不代表没法从前人经验中走捷径.别担心后面的内容会更加实用.
